In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time

import verbose_module
sys.path.append('../../.')
import pIRPgym


########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']
look_ahead = ['d','q']


# Historical data
historical_data = ['*']


# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [6,20]}          # Offer
p_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}

d_params = {'dist': 'log-normal', 'r_f_params': [3,1]}          # Demand

h_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}         # Holding costs

disc = ("strong","conc")


### Environment 
# Creating environment object
routing = True
inventory = True
perishability = 'ages'
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-01


# M = 12, K = 12

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':12,'K':12,'T':8,'F':14,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 15       # Random seeds
det_rd_seed = 35

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    10	| 6.78	  57 	   3 	6007.6 	| 13.58	   3 	  52 	   3 	5999.6 	| 13.42	   6 	  38 	   3 	5999.6 	| 12.84	   85 	  13 	   3 	5999.6 	| 12.42	   287 	  10 	   3 	6000.1 	|
1    8	| 0.64	  24 	   3 	5852.4 	| 5.75	   5 	  26 	   3 	5852.3 	| 5.70	   7 	  26 	   3 	5852.3 	| 6.72	 

# M = 15, K = 15

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':15,'K':15,'T':8,'F':14,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 15       # Random seeds
det_rd_seed = 35

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    11	| 57.33	  104 	   2 	4401.3 	| 132.9	   2 	  66 	   2 	4401.3 	| 110.4	   16 	  72 	   2 	4401.3 	| 97.08	   85 	  43 	   2 	4401.3 	| 121.1	   1563 	  11 	   2 	4401 	|
1    10	| 24.62	  58 	   2 	4478.6 	| 48.64	   6 	  40 	   2 	4478.6 	| 47.95	   9 	  40 	   2 	4478.6 	| 

AttributeError: Unable to retrieve attribute 'x'

# alpha = 0.5

In [ ]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':12,'K':12,'T':8,'F':12,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 15       # Random seeds
det_rd_seed = 35

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)
    CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,
                                                                                                    verbose=False,heuristic_initialization=True,
                                                                                                    return_num_cols=True,RCL_alpha=0.5)       # Column Generation algorithm                  
    if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=True)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


## (M = 10, K = 14, T = 12)

In [ ]:
################################## Policy Evaluation ##################################
env_config = {'M':10,'K':14,'T':12,'F':7,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 0       # Random seeds
det_rd_seed = 0

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols,len(CG_routes),CG_obj)
    CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,verbose=False,heuristic_initialization=True,return_num_cols=True)       # Column Generation algorithm                  
    if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=True)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')

************  Evaluation of Heuristic Initialization on CG  ***********
--------|	  	CG 	 	|	    CG w/ Init	 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	 cols	 #Veh 	 Obj 	|
-------------------------------------------------------------------------
0    10	| 5.86	 53 	3 	4982.6 	| 9.80	 77 	3 	4982.6 	|
1    9	| 2.18	 32 	3 	4980.9 	| 7.24	 39 	3 	4980.9 	|
2    10	| 7.18	 49 	3 	4982.6 	| 10.11	 80 	3 	4982.6 	|
3    10	| 6.74	 53 	3 	4982.6 	| 11.13	 79 	3 	4982.6 	|
4    9	| 2.26	 32 	3 	4980.9 	| 7.55	 39 	3 	4980.9 	|
5    10	| 8.60	 49 	3 	4982.6 	| 11.24	 80 	3 	4982.6 	|
6    8	| 1.25	 16 	3 	4767.8 	| 7.81	 32 	3 	4767.8 	|
7    9	| 2.61	 32 	3 	4980.9 	| 7.84	 39 	3 	4980.9 	|
8    4	| 0.09	 7 	2 	2343.3 	| 5.09	 10 	2 	2343.3 	|
9    9	| 8.69	 47 	3 	4797.2 	| 10.28	 58 	3 	4797.2 	|
10    7	| 1.19	 21 	3 	4189.6 	| 6.06	 31 	3 	4189.1 	|
11    6	| 0.60	 19 	2 	2948.1 	| 5.64	 20 	2 	2948.1 	|
Finished episode!!!


## (M = 15, K = 15, T = 12)

In [ ]:
################################## Policy Evaluation ##################################
env_config = {'M':15,'K':15,'T':12,'F':7,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 21       # Random seeds
det_rd_seed = 31

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols,len(CG_routes),CG_obj)
    CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,verbose=False,heuristic_initialization=True,return_num_cols=True)       # Column Generation algorithm                  
    if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=True)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')

************  Evaluation of Heuristic Initialization on CG  ***********
--------|	  	CG 	 	|	    CG w/ Init	 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	 cols	 #Veh 	 Obj 	|
-------------------------------------------------------------------------


KeyboardInterrupt: 

# Large instance (M = 20, K = 20, T = 8)

In [ ]:
################################## Policy Evaluation ##################################
env_config = {'M':20,'K':20,'T':12,'F':7,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 22       # Random seeds
det_rd_seed = 32

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols,len(CG_routes),CG_obj)
    CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,verbose=False,heuristic_initialization=True,return_num_cols=True)       # Column Generation algorithm                  
    if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=True)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')